# Credit Risk Resampling Techniques

In [4]:
import warnings
warnings.filterwarnings('ignore')

In [5]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

# Read the CSV into DataFrame

In [6]:
# Load the data
file_path = Path('Resources/LoanStats_2019Q1.csv')
loan_stats_df = pd.read_csv(file_path)
loan_stats_df.head()

# Drop null columns 
loan_stats_df = loan_stats_df.dropna(axis='columns', how='all')

# Drop null rows
loan_stats_df = loan_stats_df.dropna()

# Remove the `Issued` loan status
issued_mask = loan_stats_df['loan_status'] != 'Issued'
loans_stats_df = loan_stats_df.loc[issued_mask]

# convert interest rate to numerical
loan_stats_df['int_rate'] = loan_stats_df['int_rate'].replace('%', '')
loan_stats_df['int_rate'] = loan_stats_df['int_rate'].astype('float') / 100

# Convert target column values to low_risk and high_risk 
x = {'Current': 'low_risk'}   
loan_stats_df = loan_stats_df.replace(x)
loan_stats_df.reset_index(inplace=True, drop=True)

#I added 'Charged Off' into high risk category according to raw data
x = dict.fromkeys(['Late (31-120 days)', 'Late (16-30 days)', 'Default', 'In Grace Period'], 'high_risk')    
loan_stats_df = loan_stats_df.replace(x)

loan_stats_df.reset_index(inplace=True, drop=True)

loan_stats_df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.001719,375.35,RENT,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.002000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.002000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.001640,353.55,RENT,92000.0,Verified,Mar-2019,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.001474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


# Split the Data into Training and Testing

In [7]:
# Define features set
x_cols = [i for i in loan_stats_df.columns if i not in ('loan_status','Unnamed: 0')]
X = loan_stats_df[x_cols]

# Create our target
y = loan_stats_df['loan_status']

In [8]:
X.describe()

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
count,68817.000000,68817.000000,68817.000000,6.881700e+04,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,...,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.0,6.881700e+04,6.881700e+04,68817.000000,6.881700e+04
mean,16677.594562,0.001277,480.652863,8.821371e+04,21.778153,0.217766,0.497697,12.587340,0.126030,17604.142828,...,0.052138,2.219423,95.057627,30.626217,0.125972,0.0,2.100332e+05,6.133843e+04,29734.128558,5.572240e+04
std,10277.348590,0.000481,288.062432,1.155800e+05,20.199244,0.718367,0.758122,6.022869,0.336797,21835.880400,...,0.390633,1.897432,8.326426,33.631463,0.336732,0.0,1.928088e+05,5.738798e+04,26795.394232,5.095845e+04
min,1000.000000,0.000600,30.890000,4.000000e+01,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,...,0.000000,0.000000,20.000000,0.000000,0.000000,0.0,3.600000e+03,2.350000e+02,100.000000,1.270000e+02
25%,9000.000000,0.000881,265.730000,5.000000e+04,13.890000,0.000000,0.000000,8.000000,0.000000,6293.000000,...,0.000000,1.000000,93.000000,0.000000,0.000000,0.0,6.697700e+04,2.650300e+04,11600.000000,2.288000e+04
50%,15000.000000,0.001180,404.560000,7.300000e+04,19.760000,0.000000,0.000000,11.000000,0.000000,12068.000000,...,0.000000,2.000000,100.000000,20.000000,0.000000,0.0,1.467100e+05,4.535700e+04,22100.000000,4.200000e+04
75%,24000.000000,0.001557,648.100000,1.040000e+05,26.660000,0.000000,1.000000,16.000000,0.000000,21735.000000,...,0.000000,3.000000,100.000000,50.000000,0.000000,0.0,3.036400e+05,7.657000e+04,39300.000000,7.249900e+04
max,40000.000000,0.003084,1676.230000,8.797500e+06,999.000000,18.000000,5.000000,72.000000,4.000000,587191.000000,...,18.000000,19.000000,100.000000,100.000000,4.000000,0.0,3.292782e+06,1.295455e+06,509400.000000,1.426964e+06


In [9]:
# Check the balance of our target values
from collections import Counter
Counter(y)

Counter({'low_risk': 68470, 'high_risk': 347})

In [10]:
# Create X_train, X_test, y_train, y_test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=78)

In [11]:
Counter(y_train)

Counter({'low_risk': 51357, 'high_risk': 255})

## Data Pre-Processing

Scale the training and testing data using the `StandardScaler` from `sklearn`. Remember that when scaling the data, you only scale the features data (`X_train` and `X_testing`).

In [12]:
# Create the StandardScaler instance
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [13]:
# Fit the Standard Scaler with the training data
# When fitting scaling functions, only train on the training dataset
X_train_scaled = scaler.fit_transform(X_train.select_dtypes(np.number))
X_test_scaled = scaler.fit_transform(X_test.select_dtypes(np.number))

In [14]:
X_train_scaled

array([[ 0.32249399,  0.58233445,  0.00394146, ...,  2.51432489,
        -0.50749786,  2.59232724],
       [-1.5281805 , -0.20011077, -1.55678958, ..., -0.7064106 ,
        -1.00170285, -0.6542185 ],
       [-0.65154521,  0.46818462, -0.84409667, ...,  0.15477763,
        -0.79204012,  0.40256525],
       ...,
       [ 0.61470575, -0.50520214,  0.04088225, ...,  0.35177318,
        -0.83696785,  0.54003559],
       [ 0.71210967, -0.50520214,  0.11535516, ...,  2.64540022,
        -0.18925979,  3.39205707],
       [-1.13856482, -0.42425953, -1.10487361, ..., -0.07967298,
        -0.93805524, -0.05616392]])

# Simple Logistic Regression

In [15]:
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state = 55)
X_resampled, y_resampled = ros.fit_resample(X_train_scaled, y_train)
Counter(y_resampled)

Counter({'low_risk': 51357, 'high_risk': 51357})

In [16]:

# Train the Logistic Regression model using the resampled data
from sklearn.linear_model import LogisticRegression
logistic = LogisticRegression(solver='lbfgs', random_state=55)
logistic.fit(X_resampled,y_resampled)
y_pred_logistic = logistic.predict(X_test_scaled)

In [17]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
score_logistic_ros = balanced_accuracy_score(y_test,y_pred_logistic)
print(score_logistic_ros)

0.7046000497968745


In [18]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix_logistic_ros = confusion_matrix(y_test,y_pred_logistic)
print(confusion_matrix_logistic_ros)

[[   59    33]
 [ 3972 13141]]


In [19]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
classification_report_logistic_ros = classification_report_imbalanced(y_test, y_pred_logistic)
print(classification_report_logistic_ros)

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.01      0.64      0.77      0.03      0.70      0.49        92
   low_risk       1.00      0.77      0.64      0.87      0.70      0.50     17113

avg / total       0.99      0.77      0.64      0.86      0.70      0.50     17205



# Oversampling

In this section, you will compare two oversampling algorithms to determine which algorithm results in the best performance. You will oversample the data using the naive random oversampling algorithm and the SMOTE algorithm. For each algorithm, be sure to complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

### Naive Random Oversampling

In [20]:
# Resample the training data with the RandomOversampler
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state = 55)
X_resampled, y_resampled = ros.fit_resample(X_train_scaled, y_train)
Counter(y_resampled)

Counter({'low_risk': 51357, 'high_risk': 51357})

In [21]:
# Train the Logistic Regression model using the resampled data
from sklearn.linear_model import LogisticRegression
model =  LogisticRegression(solver='lbfgs', random_state=1, max_iter=100)
model = model.fit(X_resampled, y_resampled)
predictions = model.predict(X_test_scaled)

In [22]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
balanced_accuracy_score(y_test, predictions)

0.7046000497968745

In [23]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
confuse = confusion_matrix(y_test, predictions)
confuse = pd.DataFrame(
    confuse, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"]
)
confuse

,Predicted 0,Predicted 1
Actual 0,59,33
Actual 1,3972,13141


In [24]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, predictions))


                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.01      0.64      0.77      0.03      0.70      0.49        92
   low_risk       1.00      0.77      0.64      0.87      0.70      0.50     17113

avg / total       0.99      0.77      0.64      0.86      0.70      0.50     17205



### SMOTE Oversampling

In [25]:
# Resample the training data with SMOTE
from imblearn.over_sampling import SMOTE

X_resampled, y_resampled = SMOTE(random_state=1).fit_resample(X_train_scaled, y_train)

# View the count of target classes with Counter
Counter(y_resampled)

Counter({'low_risk': 51357, 'high_risk': 51357})

In [26]:
# Train the Logistic Regression model using the resampled data
model_smote = LogisticRegression(solver='lbfgs', random_state = 1)
model_smote = model_smote.fit(X_resampled, y_resampled)
predictions_smote = model_smote.predict(X_test_scaled)

In [27]:
# Calculated the balanced accuracy score
balanced_accuracy_score(y_test, predictions_smote)

0.6807864730347384

In [28]:
# Display the confusion matrix
confuse = confusion_matrix(y_test, predictions_smote)
confuse_df = pd.DataFrame(
    confuse, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"]
)
confuse

array([[   55,    37],
       [ 4043, 13070]])

In [29]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, predictions_smote))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.01      0.60      0.76      0.03      0.68      0.45        92
   low_risk       1.00      0.76      0.60      0.86      0.68      0.46     17113

avg / total       0.99      0.76      0.60      0.86      0.68      0.46     17205



# Undersampling

In this section, you will test an undersampling algorithm to determine which algorithm results in the best performance compared to the oversampling algorithms above. You will undersample the data using the Cluster Centroids algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Display the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [ ]:
# Resample the data using the ClusterCentroids resampler
from imblearn.under_sampling import ClusterCentroids
cc = ClusterCentroids(random_state=1)
X_resampled, y_resampled = cc.fit_resample(X_train_scaled, y_train)

# View the count of target classes with Counter
Counter(y_resampled)

In [ ]:
# Train the Logistic Regression model using the resampled data
cc_model = LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='auto',
          n_jobs=None, penalty='l2', random_state=1, solver='saga',
          tol=0.0001, verbose=0, warm_start=False)

cc_model.fit(X_resampled, y_resampled)
prediction_cc = cc_model.predict(X_test_scaled)

In [ ]:
# Calculate the balanced accuracy score
balanced_accuracy_score(y_test, prediction_cc)

In [ ]:
# Display the confusion matrix
cm = confusion_matrix(y_test, prediction_cc)
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"]
)
cm

In [ ]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, prediction_cc))

In [ ]:
# Train the Logistic Regression model using the resampled data
smoteen_model = LogisticRegression(solver='lbfgs', random_state=1)
smoteen_model.fit(X_resampled, y_resampled)
prediction_smoteen = model.predict(X_test)

# Combination (Over and Under) Sampling

In this section, you will test a combination over- and under-sampling algorithm to determine if the algorithm results in the best performance compared to the other sampling algorithms above. You will resample the data using the SMOTEENN algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Display the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [ ]:
# Resample the training data with SMOTEENN
from imblearn.combine import SMOTEENN

smoteen = SMOTEENN(random_state=1)

# View the count of target classes with Counter
X_resampled, y_resampled = smoteen.fit_resample(X_train_scaled, y_train)
Counter(y_resampled)

In [ ]:
# Training the Logistic Regression model using the resampled data
smoteen_model = LogisticRegression(solver='lbfgs', random_state=1)
smoteen_model.fit(X_resampled, y_resampled)
prediction_smoteen = model.predict(X_test)

In [ ]:
# Calculate the balanced accuracy score
balanced_accuracy_score(y_test, prediction_smoteen)

In [ ]:
# Display the confusion matrix
cm = confusion_matrix(y_test, prediction_smoteen)
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"]
)
cm

In [ ]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, prediction_smoteen))

# Final Questions

1. Which model had the best balanced accuracy score?

   Random sampling method

2. Which model had the best recall score?

    Clustered Centroids.

3. Which model had the best geometric mean score?

    Both random sampling methods.
